<a href="https://colab.research.google.com/github/ILTRENTA/CTGAIN/blob/main/CTGAIN_testing_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ILTRENTA/CTGAIN.git

!pip install -r CTGAIN/requirements_ctgain.txt


Cloning into 'CTGAIN'...
remote: Enumerating objects: 200, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 200 (delta 33), reused 66 (delta 25), pack-reused 112
Receiving objects: 100% (200/200), 2.53 MiB | 14.87 MiB/s, done.
Resolving deltas: 100% (68/68), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining rdt from git+https://github.com/ILTRENTA/RDT.git@2cf799b7c305430b9932fd90c45283492c8f4aeb#egg=rdt (from -r CTGAIN/requirements_ctgain.txt (line 46))
  Cloning https://github.com/ILTRENTA/RDT.git (to revision 2cf799b7c305430b9932fd90c45283492c8f4aeb) to ./src/rdt
  Running command git clone --filter=blob:none --quiet https://github.com/ILTRENTA/RDT.git /content/src/rdt
  Running command git rev-parse -q --verify 'sha^2cf799b7c305430b9932fd90c45283492c8f4aeb'
  Running command git fetch -q https://github.com/ILTRENTA/RDT.git 2cf799b7c305430b9932fd90c452834

# IMPORTANT: You must restart your runtime before running the code below, otherwise it won't work

In [ ]:
import os 

os.chdir("CTGAIN")
import torch 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.datasets import load_wine, load_breast_cancer
from ctgain.ctgain_v2 import CTGAIN_v2 as CTGAIN

from ctgain.utils.ctgain import *
from ctgain.utils.GAIN_metrics import *

In [ ]:
dataset_name="abalone.csv"#@param  ["abalone.csv" , "  diamonds.csv" , " hotel.csv" ,"red-wine.csv","concrete.csv" ,"fuel.csv" , "     housing.csv" , " Letter.csv"]




In [ ]:
data=pd.read_csv(f"/content/CTGAIN/datasets/{dataset_name}.csv")
print(data.head())

cols=data.columns

Mechanism="MCAR"#@param ["MCAR", "MAR"]

# p_miss and p_obs should always add <1 otherwise produce_NA will raise an error
# If MCAR is the mechanism, p_obs doesn't affect the missingness 
trans_dta=produce_NA(data.values, p_miss=.2, 
                     p_obs=.5, mecha=Mechanism)



incomp_dta=trans_dta["X_incomp"]
print(incomp_dta.shape)

incomp_dta=pd.DataFrame(incomp_dta, columns=cols)
incomp_dta.head()


model=CTGAIN(epochs=1000, #verbose=True, The current implementation includes a tqdm loading bar which resutls in an ugly output hence keep verbose =False
              batch_size=256,
             hint_rate=0.9, # hint rate helps the discriminator with guessing the mask matrix
             alpha=1)# Alpha in the experiment is set also to 26 


model.fit(incomp_dta)
display(pd.DataFrame(trans_dta["X_init"],columns=cols))
display(incomp_dta)

mask=1-incomp_dta.isnull().astype(int).values

#model.impute can return can also output the estimates of the observed data
# if real_only=True  the model returns the original data + the imputation 
# if real_only=False then it outputs the generated sample data with the imputation in it
#imputed, _, _,=model.impute(incomp_dta.copy(), real_only=False)

imputed=model.impute(incomp_dta.copy())
display(imputed)
print(rmse_loss(data.values,imputed.values ,mask))


   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  MedHouseVal  
0    -122.23        4.526  
1    -122.22        3.585  
2    -122.24        3.521  
3    -122.25        3.413  
4    -122.25        3.422  
torch.Size([20640, 9])
2
Epoch 1, Loss G:  69.8866, Loss D:  0.4927  g_loss mse comp   2.6859  0.0524
Epoch 2, Loss G:  68.5427, Loss D:  0.4851  g_loss mse comp   2.6342  0.0532
Epoch 3, Loss G:  66.6349, Loss D:  0.4803  g_loss mse comp   2.5609  0.0511
Epoch 4, Loss G:  65.0951, Loss D:  0.4911  g_loss mse comp   2.5018  0.0487
Epoch 5, Loss G:  67.5392, Loss D:  0.5117